<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/2025-%20Additional%20Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [371]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import plotly.graph_objects as go
import keras
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed, Input
from keras.models import Model
from keras import saving

import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import EarlyStopping
from keras.callbacks import Callback
import plotly
from keras import losses
import plotly.express as px # for data visualization
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
import numpy as np
import scipy.stats as stats
import pylab as pl



##### Take one dataset at a time
#train_data = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_METROPM/multivariate_long_sequences-TRAIN-Daily-DIRECT-VAR.npy')
#train_data = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_AIRQUALITY/multivariate_long_sequences-TRAIN-Daily-DIRECT-VAR.npy')
train_data = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_ELECTRICITY/multivariate_long_sequences-TRAIN-DATA-FullSearch-VAR.npy')


n_seq = train_data.shape[0]
window_size = train_data.shape[1]
n_features = train_data.shape[2]

##For VAR
#TestSample = 10

##For DyforGP
TestSample = 1


##Get large set of subsequence from previous steps (VAE)... this is not labelled yet.
#results1=np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_METROPM/generated_large_subsquence2_data_V2.npy')
#results1=np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_AIRQUALITY/generated_large_subsquence2_data.npy')
results1=np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_ELECTRICITY/generated_large_subsquence2_data.npy')

x=results1




############### VAR PREDICTION TIMING ####################################################################################################################

!pip install ipython-autotime
##Start measuring time-----THIS ONE IS FOR VAR --------------------------------
%load_ext autotime

rmse_list = list()
min_window_list = list()
best_window_for_long_seq = list()
best_window_for_long_seq.clear()
#for i in range(2) :
x_3d = x.reshape((x.shape[0],window_size,n_features))
n_future = 1
K= window_size

import random
rd = list()
rd = random.sample(range(1, x_3d.shape[0]), TestSample)


best_window_for_long_seq.clear()

from statsmodels.tsa.api import VAR

## Note for some reason first run takes longer. So run it twice to get a better time
for i in rd :

  rmse_list.clear()
  for k in range(2,(round(K))):
    cur_seq = x_3d[i]
    df = pd.DataFrame(cur_seq, columns=['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12'])
    #df = pd.DataFrame(cur_seq, columns=['V1','V2','V3','V4','V5','V6'])
    df_train, df_test = df[0:-n_future], df[-n_future:]
    model= VAR(df_train)
    try:
      model_fitted1 = model.fit(k)
      forecast_input1 = df_train.values[-k:]
      fc1 = model_fitted1.forecast(y=forecast_input1, steps=n_future)
      df_forecast1 = pd.DataFrame(fc1, index=df.index[-n_future:], columns=df.columns)
      mse =  mean_squared_error(df_test['V1'], df_forecast1['V1'].values)
      rmse_list.append(mse)
    except:
      rmse_list.append(99999)
      print('VAR could not solve row number')
      print(i, k)

  #For this i, find minimum rmse for all sliding window, and corresponding sw size
  min_index = rmse_list.index(min(rmse_list))
  min_sw = min_index + 2
  print('i=', i,'SW =', min_sw, rmse_list)
  best_window_for_long_seq.append(min_sw)


#OUT OF ALL LOOPS NOW
#best_window_for_long_seq now contains the best multivariate window size for each of the long sequence i
print(best_window_for_long_seq)
Window = np.array(best_window_for_long_seq)
y= Window

##Commenting this for now but they may be needed
#np.save(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_METROPM/generated-data-true-window2-COHORT1.npy',y)
#np.save(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_METROPM/generated-data2-COHORT1.npy',x)


########################## MLP PREDICTION TIMING ###################################################################################
#---------------------------------------------------------Get parts of the results and combine them --------------------------------


#y = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_METROPM/generated-data-true-window2.npy')
#x = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_METROPM/generated-data2.npy')

#y = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_AIRQUALITY/generated-data-true-window2.npy')
#x = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_AIRQUALITY/generated-data2.npy')

x=np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_ELECTRICITY/generated-data2.npy')
y=np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_ELECTRICITY/generated-data-true-window2.npy')

from sklearn.preprocessing import PowerTransformer
transformer = StandardScaler()
y_transformed = transformer.fit_transform(y.reshape(-1,1)).flatten()

##Simply take first 100 records - no need to sample... we just need to check prediction time

x_test = x[rd]
y_test = y_transformed[rd]
#------------------------------------------------------Retrieve Trained MLP Model----------

#base_model = keras.models.load_model(r'/content/drive/MyDrive/PHD/2024/DGRNet-MLP-Versions/METROPM_MLP_model_Daily.keras')
#base_model = keras.models.load_model(r'/content/drive/MyDrive/PHD/2024/DGRNet-MLP-Versions/AQ_MLP_model_Daily.keras')
base_model = keras.models.load_model(r'/content/drive/MyDrive/PHD/2024/DGRNet-MLP-Versions/ELEC_MLP_model_Daily.keras')

#-----------------------------------------------------Run Prediction on MLP------------------------
## Note for some reason first run takes longer. So run it twice to get a better time

y_pred_raw = base_model.predict(x_test)
y_test_pred = transformer.inverse_transform(y_pred_raw)
print(y_test_pred)

############### DyFor GP PREDICTION TIMING ####################################################################################################################
############### VAR PREDICTION TIMING ####################################################################################################################


##############DyFOR GP Heuristic#######################################################
from statsmodels.tsa.arima.model import ARIMA
n_future = n_features

def get_window_size (series, pred_point, small_w, large_w):
  # pred_point ==> the point of prediction (starting point)
  # small_w ==> smaller window size preceding the pred_point
  # large_w ==> larger window size preceding the pred_point
  # series is flttened long subsequence --> multivariate converted into univariate by flattening...


  pred_l_start = pred_point - large_w
  pred_s_start = pred_point - small_w
  pred_l_window = series[pred_l_start:pred_point]
  pred_s_window = series[pred_s_start:pred_point]
  modell = ARIMA(pred_l_window, order=(2,0,0))
  fitl = modell.fit()
  forecastl = fitl.forecast(steps=n_future)
  models = ARIMA(pred_s_window, order=(2,0,0))
  fits = models.fit()
  forecasts = fits.forecast(steps=n_future)
  mse_l = mean_squared_error(series[pred_point:pred_point+n_future], forecastl)
  mse_s = mean_squared_error(series[pred_point:pred_point+n_future], forecasts)
  if mse_l < mse_s:
    return large_w
  else:
    return small_w


best_window_for_long_seq = list()
best_window_for_long_seq.clear()


#for i in range(x.shape[0]):
for i in rd:
  print("starting sequence: ", i, " out of ", x.shape[0])
  cur_seq = x[i,:]
  start_small_w = 2
  start_large_w = 3
  start_index = start_large_w
  ### we need to increment by number of variables, as we have flattened the ime series
  for k in range (start_index*n_features, x.shape[1]-n_features+1, n_features):
    print("starting point: ", k, " out of ", x.shape[1]-n_features)
    pred_point_k = k
    small_w_k = start_small_w*n_features
    large_w_k = start_large_w*n_features
    print("small_w_k: ", small_w_k, " large_w_k: ", large_w_k)
    best_window_k = get_window_size(cur_seq, pred_point_k, small_w_k, large_w_k)
    print("best_window_k: ", best_window_k)
    if best_window_k == large_w_k:
      start_large_w = start_large_w + 1
      start_small_w = start_small_w + 1
    if best_window_k == small_w_k:
      if start_small_w > 1:
        start_large_w = start_large_w -1
        start_small_w = start_small_w -1
      else:
        start_large_w = start_large_w
        start_small_w = start_small_w
  print("getting best window for this sequenc",  i, "small",small_w_k, "large= ",large_w_k, "best window= ", best_window_k)
  #final_best_window =  get_window_size(cur_seq, k, small_w_k, large_w_k)
    #print("final_best_window: ", final_best_window)
  best_window_for_long_seq.append(best_window_k)


print(best_window_for_long_seq)
Window = np.array(best_window_for_long_seq)/n_features


starting sequence:  112591  out of  136300
starting point:  18  out of  174
small_w_k:  12  large_w_k:  18
best_window_k:  12
starting point:  24  out of  174
small_w_k:  6  large_w_k:  12
best_window_k:  12
starting point:  30  out of  174
small_w_k:  12  large_w_k:  18
best_window_k:  18
starting point:  36  out of  174
small_w_k:  18  large_w_k:  24
best_window_k:  24
starting point:  42  out of  174
small_w_k:  24  large_w_k:  30
best_window_k:  30
starting point:  48  out of  174
small_w_k:  30  large_w_k:  36
best_window_k:  36
starting point:  54  out of  174
small_w_k:  36  large_w_k:  42
best_window_k:  42
starting point:  60  out of  174
small_w_k:  42  large_w_k:  48
best_window_k:  48
starting point:  66  out of  174
small_w_k:  48  large_w_k:  54
best_window_k:  54
starting point:  72  out of  174
small_w_k:  54  large_w_k:  60
best_window_k:  60
starting point:  78  out of  174
small_w_k:  60  large_w_k:  66
best_window_k:  60
starting point:  84  out of  174
small_w_k:  

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


best_window_k:  90
starting point:  150  out of  174
small_w_k:  84  large_w_k:  90
best_window_k:  84
starting point:  156  out of  174
small_w_k:  78  large_w_k:  84
best_window_k:  84
starting point:  162  out of  174
small_w_k:  84  large_w_k:  90
best_window_k:  90
starting point:  168  out of  174
small_w_k:  90  large_w_k:  96
best_window_k:  96
starting point:  174  out of  174
small_w_k:  96  large_w_k:  102
best_window_k:  102
getting best window for this sequenc 112591 small 96 large=  102 best window=  102
[102]
time: 3.43 s (started: 2025-03-25 15:18:34 +00:00)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')